# **Uploading the files and concating the DFs**

In [1]:
import pandas as pd
import numpy as np
import re
import statistics

In [2]:
letters = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','z']

In [3]:
#Building a dicto_scraped with all DF to be able to concat everything
dicto_scraped = {letter : pd.read_csv("../BEEV/letter_scraped/"+ letter +"_scraped.csv",index_col=None) for letter in letters}

C:\Users\Utilizador\anaconda3\envs\Safe space\lib\site-packages\IPython\core\interactiveshell.py:3364: DtypeWarning: Columns (56,80,95,110,111,112,116,121,122,123,127,128,129,130,131,132,133,134,135,136,137,138,139,142) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
C:\Users\Utilizador\anaconda3\envs\Safe space\lib\site-packages\IPython\core\interactiveshell.py:3364: DtypeWarning: Columns (55,56,108,118,119,123,124,125,126,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,149,150,151,152,153,154,155,156,157,158,159,160,167,168,169,170,171) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [4]:
#Building a list of shape to check coherence between letter.csv and letter_scraped_csv
scraped_shape = [dicto_scraped[let].shape[0] for let in dicto_scraped]

In [5]:
#Building a dicto_csv with all url list to be able to compare with dicto_scraped
dicto_csv = {letter : pd.read_csv("../BEEV/letter_csvs/"+ letter +".csv",index_col=None) for letter in letters}

In [6]:
#Building a list of shape to check coherence between letter.csv and letter_scarped_csv
csv_shape = [dicto_csv[lett].shape[0] for lett in dicto_csv.keys()]

In [7]:
#Calculating the difference between both lenght list
scraped_shape
for i in range(len(scraped_shape)):
    print(scraped_shape[i] - csv_shape[i])  

-14
-13
-14
-6
0
-21
-5
-15
-3
-4
-7
-7
-35
-11
-3
-7
0
-7
-10
-18
0
-10
-7
0
0


In [8]:
#The difference come from the empty dataframe we scrapped so we have to count the number of empty DF to see if it macth
# test calculation --->  
#dicto_csv['a']['dataframe'][dicto_csv['a']['dataframe'].str.contains('Empty') == True].shape[0]
empty_df_list = [dicto_csv[letter]['dataframe'][dicto_csv[letter]['dataframe'].str.contains('Empty') == True].shape[0] for letter in letters]
    

In [9]:
#Calculating the difference between both lenght list and the number of empty DF
# If every number is equal to 0 we are ok ! next step will be to check the model coherence
for i in range(len(scraped_shape)):
    print(scraped_shape[i] - csv_shape[i] + empty_df_list[i])  

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


# **GEV dataframe**

In [10]:
#Building the DF using the DF in the dicto_scraped
df = pd.concat([dicto_scraped[letter] for letter in letters])

In [11]:
df_gev = df[df['Powertrain Architecture'] == 'Internal Combustion engine'].copy() #The GEV cars are all the ones with Internal combustion engine
df_gev.reset_index(drop=False, inplace=True)
df_gev.drop(columns = ['index'], inplace =True)

In [12]:
#Definition of the key word for filtering columns
words_wanted4_column = ['Brand','Model','Generation', 'Engine', 'production','Powertrain','Body','Fuel','CO2','Power']

In [13]:
#Checking manually to see which key words we need for the columns
#df_gev.columns[79:110]
len(list(df_gev.columns))

192

In [14]:
#Building the list of columns containing the key words
col_filtered = []
for word in words_wanted4_column:
  for w in list(df_gev.columns):
    if w.lower().find(word.lower()) != -1:
      col_filtered.append(w)
len(col_filtered)

94

In [15]:
#filtering fuel columns
fuel_list =['Fuel consumption','fuel consumption']
fuel_col_filtered = []
for word in fuel_list:
  for w in col_filtered:
    if w.find(word) != -1:
      fuel_col_filtered.append(w)
len(fuel_col_filtered)
fuel_col_filtered

['Fuel consumption (economy) - urban',
 'Fuel consumption (economy) - extra urban',
 'Fuel consumption (economy) - combined',
 'Fuel consumption (economy) - urban (NEDC)',
 'Fuel consumption (economy) - extra urban (NEDC)',
 'Fuel consumption (economy) - combined (NEDC)',
 'Fuel consumption at Low speed (WLTP)',
 'Fuel consumption at Medium speed (WLTP)',
 'Fuel consumption at high speed (WLTP)',
 'Fuel consumption at very high speed (WLTP)',
 'Fuel consumption (economy) - urban (LPG)',
 'Fuel consumption (economy) - extra urban (LPG)',
 'Fuel consumption (economy) - combined (LPG)',
 'Fuel consumption (economy) - urban (NEDC, WLTP equivalent)',
 'Fuel consumption (economy) - extra urban (NEDC, WLTP equivalent)',
 'Fuel consumption (economy) - combined (NEDC, WLTP equivalent)',
 'Fuel consumption at Low speed (WLTP) (CNG)',
 'Fuel consumption at Medium speed (WLTP) (CNG)',
 'Fuel consumption at high speed (WLTP) (CNG)',
 'Fuel consumption at very high speed (WLTP) (CNG)',
 'Fuel consum

## CO2 cleaning

In [16]:
#filtering CO2 columns
co2_list =['CO2 emissions']
co2_col_filtered = []
for word in co2_list:
  for w in col_filtered:
    if w.find(word) != -1:
      co2_col_filtered.append(w)
len(co2_col_filtered)
print(co2_col_filtered)
co2_col_filtered[3].split()

['CO2 emissions', 'CO2 emissions (NEDC)', 'CO2 emissions (WLTP)', 'CO2 emissions (LPG)', 'CO2 emissions (NEDC, WLTP equivalent)', 'CO2 emissions (CNG)', 'CO2 emissions (CNG) (NEDC)', 'CO2 emissions (Ethanol - E85)', 'CO2 emissions (LPG) (NEDC)', 'CO2 emissions (LPG) (NEDC, WLTP equivalent)', 'CO2 emissions (LPG) (WLTP)', 'CO2 emissions (CNG) (WLTP)', 'CO2 emissions (CNG) (NEDC, WLTP equivalent)']


['CO2', 'emissions', '(LPG)']

In [ ]:
#Grabbing the interesting part in the column name
for w in co2_col_filtered:
    print(" ".join(w.split()[2:]))

def co2_grab(dfserie):
    return " ".join(dfserie.name.split()[2:])

In [18]:
df_gev[['CO2 emissions',
 'CO2 emissions (NEDC)',
 'CO2 emissions (WLTP)',
 'CO2 emissions (LPG)',
 'CO2 emissions (NEDC, WLTP equivalent)',
 'CO2 emissions (CNG)',
 'CO2 emissions (CNG) (NEDC)',
 'CO2 emissions (Ethanol - E85)',
 'CO2 emissions (LPG) (NEDC)',
 'CO2 emissions (LPG) (NEDC, WLTP equivalent)',
 'CO2 emissions (LPG) (WLTP)',
 'CO2 emissions (CNG) (WLTP)',
 'CO2 emissions (CNG) (NEDC, WLTP equivalent)']]#.isna().sum()

,CO2 emissions,CO2 emissions (NEDC),CO2 emissions (WLTP),CO2 emissions (LPG),"CO2 emissions (NEDC, WLTP equivalent)",CO2 emissions (CNG),CO2 emissions (CNG) (NEDC),CO2 emissions (Ethanol - E85),CO2 emissions (LPG) (NEDC),"CO2 emissions (LPG) (NEDC, WLTP equivalent)",CO2 emissions (LPG) (WLTP),CO2 emissions (CNG) (WLTP),"CO2 emissions (CNG) (NEDC, WLTP equivalent)"
0,153 g/km,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,148 g/km,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,153 g/km,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,148 g/km,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,151 g/km,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
43107,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43108,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43109,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43110,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
df_co2 = df_gev[co2_col_filtered].copy()
for column in co2_col_filtered:
    df_co2[column] = df_gev[[column]].apply(lambda item: item + co2_grab(item))
df_co2['co2_clean'] = df_co2.fillna('').sum(axis=1)

In [20]:
#df_co2['co2_clean'][200:250]
df_co2['co2_clean'] = df_co2['co2_clean'].apply(lambda item: np.nan if len(item) ==0 else item)

In [21]:
#df_co2.isna().sum()
#df_co2['co2_clean'].iloc[200].split()[0]
df_co2['co2_nb'] = df_co2['co2_clean'].apply(lambda item: np.nan if isinstance(item, float) else (item.replace('l','1').split()[0] if item.replace('l','1').split()[0].find('-') == -1 else item.split()[0].split('-')[0]))
df_co2['co2_unit'] = df_co2['co2_clean'].apply(lambda item: np.nan if isinstance(item, float) else item.split()[1])
df_co2['co2_nb'] = df_co2['co2_nb'].apply(lambda item: float(item) if (not isinstance(item, float) and item.isdigit()) else item) #.str.isnumeric()
df_co2[['co2_clean','co2_nb','co2_unit']]

,co2_clean,co2_nb,co2_unit
0,153 g/km,153.0,g/km
1,148 g/km,148.0,g/km
2,153 g/km,153.0,g/km
3,148 g/km,148.0,g/km
4,151 g/km(NEDC),151.0,g/km(NEDC)
...,...,...,...
43107,NaN,NaN,NaN
43108,NaN,NaN,NaN
43109,NaN,NaN,NaN
43110,NaN,NaN,NaN


In [22]:
df_gev_co2 = df_gev[['Brand', 'Model', 'Generation', 'Modification (Engine)', 'Body type', 'Fuel tank capacity', 'Fuel consumption (economy) - combined', 'Fuel Type', 'CO2 emissions', 'Power']].merge(df_co2[['co2_clean','co2_nb','co2_unit']],left_index=True,right_index=True)

In [23]:
df_gev_co2.head()

,Brand,Model,Generation,Modification (Engine),Body type,Fuel tank capacity,Fuel consumption (economy) - combined,Fuel Type,CO2 emissions,Power,co2_clean,co2_nb,co2_unit
0,Abarth,124 Spider,124 GT,1.4 MultiAir (170 Hp) Automatic,Roadster,45 l 11.89 US gal | 9.9 UK gal,6.6 l/100 km 35.64 US mpg42.8 UK mpg15.15 km/l,Petrol (Gasoline),153 g/km,170 Hp @ 5500 rpm.,153 g/km,153.0,g/km
1,Abarth,124 Spider,124 GT,1.4 MultiAir (170 Hp),Roadster,45 l 11.89 US gal | 9.9 UK gal,6.4 l/100 km 36.75 US mpg44.14 UK mpg15.63 km/l,Petrol (Gasoline),148 g/km,170 Hp @ 5500 rpm.,148 g/km,148.0,g/km
2,Abarth,124 Spider,124 Spider,1.4 MultiAir (170 Hp) Automatic,Cabriolet,45 l 11.89 US gal | 9.9 UK gal,6.6 l/100 km 35.64 US mpg42.8 UK mpg15.15 km/l,Petrol (Gasoline),153 g/km,170 Hp @ 5500 rpm.,153 g/km,153.0,g/km
3,Abarth,124 Spider,124 Spider,1.4 MultiAir (170 Hp),Cabriolet,45 l 11.89 US gal | 9.9 UK gal,6.4 l/100 km 36.75 US mpg44.14 UK mpg15.63 km/l,Petrol (Gasoline),148 g/km,170 Hp @ 5500 rpm.,148 g/km,148.0,g/km
4,Abarth,500,500C,1.4 T-Jet (140 Hp) MTA,Cabriolet,35 l 9.25 US gal | 7.7 UK gal,NaN,Petrol (Gasoline),NaN,140 Hp @ 5500 rpm.,151 g/km(NEDC),151.0,g/km(NEDC)


In [24]:
#Function to split the CO2 values that have extra characters
def split_bin(value):
    if value.find('-') != -1:
        value = value.split('-')
        value[0] = float(value[0])
        if len(value[1])>1:
            value[1] = float(value[1])
        else: value[1] = float(value[0])
        return value
    elif value.find('+') != -1:
        value = value.split('+')
        value[0] = float(value[0])
        value[1] = float(value[1])
        return value
    else: return value
    
#Function to split the fuel values that have extra characters
def basic_split_bin(value):
    if value.find('-') != -1:
        value = value.split('-')
        if len(value[1])==0:
            value[1] = value[0]
        return value
    elif value.find('+') != -1:
        value = value.split('+')
        return value
    else: return value

In [25]:
#Cleaning Fuel tank capacity in a new column
df_gev_co2['Fuel_tank_capacity_clean_(liters)'] = df_gev_co2['Fuel tank capacity'].apply(lambda item: item.split()[0] if (not isinstance(item, float)) else item)
df_gev_co2['Fuel_tank_capacity_clean_(liters)'] = df_gev_co2['Fuel_tank_capacity_clean_(liters)'].apply(lambda item: float(item) if (not isinstance(item, float) and item.replace('+','-').find('-') == -1) else item) #and item.isdigit()) else item) #.str.isnumeric()
df_gev_co2['Fuel_tank_capacity_clean_(liters)'] = df_gev_co2['Fuel_tank_capacity_clean_(liters)'].apply(lambda item: split_bin(item) if (not isinstance(item, float)) else item)
df_gev_co2.head()

,Brand,Model,Generation,Modification (Engine),Body type,Fuel tank capacity,Fuel consumption (economy) - combined,Fuel Type,CO2 emissions,Power,co2_clean,co2_nb,co2_unit,Fuel_tank_capacity_clean_(liters)
0,Abarth,124 Spider,124 GT,1.4 MultiAir (170 Hp) Automatic,Roadster,45 l 11.89 US gal | 9.9 UK gal,6.6 l/100 km 35.64 US mpg42.8 UK mpg15.15 km/l,Petrol (Gasoline),153 g/km,170 Hp @ 5500 rpm.,153 g/km,153.0,g/km,45.0
1,Abarth,124 Spider,124 GT,1.4 MultiAir (170 Hp),Roadster,45 l 11.89 US gal | 9.9 UK gal,6.4 l/100 km 36.75 US mpg44.14 UK mpg15.63 km/l,Petrol (Gasoline),148 g/km,170 Hp @ 5500 rpm.,148 g/km,148.0,g/km,45.0
2,Abarth,124 Spider,124 Spider,1.4 MultiAir (170 Hp) Automatic,Cabriolet,45 l 11.89 US gal | 9.9 UK gal,6.6 l/100 km 35.64 US mpg42.8 UK mpg15.15 km/l,Petrol (Gasoline),153 g/km,170 Hp @ 5500 rpm.,153 g/km,153.0,g/km,45.0
3,Abarth,124 Spider,124 Spider,1.4 MultiAir (170 Hp),Cabriolet,45 l 11.89 US gal | 9.9 UK gal,6.4 l/100 km 36.75 US mpg44.14 UK mpg15.63 km/l,Petrol (Gasoline),148 g/km,170 Hp @ 5500 rpm.,148 g/km,148.0,g/km,45.0
4,Abarth,500,500C,1.4 T-Jet (140 Hp) MTA,Cabriolet,35 l 9.25 US gal | 7.7 UK gal,NaN,Petrol (Gasoline),NaN,140 Hp @ 5500 rpm.,151 g/km(NEDC),151.0,g/km(NEDC),35.0


## Fuel cleaning

### Creating the dataframes

In [26]:
df_fuel = df_gev[['Fuel consumption (economy) - urban', 'Fuel consumption (economy) - extra urban','Fuel consumption (economy) - combined',
 'Fuel consumption (economy) - urban (NEDC)','Fuel consumption (economy) - extra urban (NEDC)','Fuel consumption (economy) - combined (NEDC)',
 'Fuel consumption (economy) - urban (LPG)','Fuel consumption (economy) - extra urban (LPG)','Fuel consumption (economy) - combined (LPG)',
 'Fuel consumption (economy) - urban (NEDC, WLTP equivalent)','Fuel consumption (economy) - extra urban (NEDC, WLTP equivalent)','Fuel consumption (economy) - combined (NEDC, WLTP equivalent)',
 'Fuel consumption (economy) - urban (CNG) (NEDC)','Fuel consumption (economy) - extra urban (CNG) (NEDC)','Fuel consumption (economy) - combined (CNG) (NEDC)',
 'Fuel consumption (economy) - urban (CNG)','Fuel consumption (economy) - extra urban (CNG)','Fuel consumption (economy) - combined (CNG)',
 'Fuel consumption (economy) - urban (Ethanol - E85)','Fuel consumption (economy) - extra urban (Ethanol - E85)','Fuel consumption (economy) - combined (Ethanol - E85)',
 'Fuel consumption (economy) - urban (LPG) (NEDC)','Fuel consumption (economy) - extra urban (LPG) (NEDC)', 'Fuel consumption (economy) - combined (LPG) (NEDC)',
 'Fuel consumption (economy) - urban (LPG) (NEDC, WLTP equivalent)','Fuel consumption (economy) - extra urban (LPG) (NEDC, WLTP equivalent)','Fuel consumption (economy) - combined (LPG) (NEDC, WLTP equivalent)',
 'Fuel consumption (economy) - urban (CNG) (NEDC, WLTP equivalent)','Fuel consumption (economy) - extra urban (CNG) (NEDC, WLTP equivalent)','Fuel consumption (economy) - combined (CNG) (NEDC, WLTP equivalent)']]

fuel_combined = ['combined']
col_combined = []
for word in fuel_combined:
  for w in list(df_fuel.columns):
    if w.lower().find(word.lower()) != -1:
      col_combined.append(w)

df_fuel_combined = df_gev[['Fuel consumption (economy) - combined',
 'Fuel consumption (economy) - combined (NEDC)',
 'Fuel consumption (economy) - combined (LPG)',
 'Fuel consumption (economy) - combined (NEDC, WLTP equivalent)',
 'Fuel consumption (economy) - combined (CNG) (NEDC)',
 'Fuel consumption (economy) - combined (CNG)',
 'Fuel consumption (economy) - combined (Ethanol - E85)',
 'Fuel consumption (economy) - combined (LPG) (NEDC)',
 'Fuel consumption (economy) - combined (LPG) (NEDC, WLTP equivalent)',
 'Fuel consumption (economy) - combined (CNG) (NEDC, WLTP equivalent)']]

#-------------------------------------------------------------------------
fuel_urban = ['urban']
col_urban = []
for word in fuel_urban:
  for w in list(df_fuel.columns):
    if w.lower().find(word.lower()) != -1 and "extra" not in w:
      col_urban.append(w)

df_fuel_urban = df_gev[['Fuel consumption (economy) - urban',
 'Fuel consumption (economy) - urban (NEDC)',
 'Fuel consumption (economy) - urban (LPG)',
 'Fuel consumption (economy) - urban (NEDC, WLTP equivalent)',
 'Fuel consumption (economy) - urban (CNG) (NEDC)',
 'Fuel consumption (economy) - urban (CNG)',
 'Fuel consumption (economy) - urban (Ethanol - E85)',
 'Fuel consumption (economy) - urban (LPG) (NEDC)',
 'Fuel consumption (economy) - urban (LPG) (NEDC, WLTP equivalent)',
 'Fuel consumption (economy) - urban (CNG) (NEDC, WLTP equivalent)']]

#----------------------------------------------------------------------------
fuel_extra_urban = ['extra urban']
col_fuel_extra_urban = []
for word in fuel_extra_urban:
  for w in list(df_fuel.columns):
    if w.lower().find(word.lower()) != -1:
      col_fuel_extra_urban.append(w)

df_fuel_extra_urban = df_gev[['Fuel consumption (economy) - extra urban',
 'Fuel consumption (economy) - extra urban (NEDC)',
 'Fuel consumption (economy) - extra urban (LPG)',
 'Fuel consumption (economy) - extra urban (NEDC, WLTP equivalent)',
 'Fuel consumption (economy) - extra urban (CNG) (NEDC)',
 'Fuel consumption (economy) - extra urban (CNG)',
 'Fuel consumption (economy) - extra urban (Ethanol - E85)',
 'Fuel consumption (economy) - extra urban (LPG) (NEDC)',
 'Fuel consumption (economy) - extra urban (LPG) (NEDC, WLTP equivalent)',
 'Fuel consumption (economy) - extra urban (CNG) (NEDC, WLTP equivalent)']]

### Combined

In [27]:
df_fuel_combined.head()

,Fuel consumption (economy) - combined,Fuel consumption (economy) - combined (NEDC),Fuel consumption (economy) - combined (LPG),"Fuel consumption (economy) - combined (NEDC, WLTP equivalent)",Fuel consumption (economy) - combined (CNG) (NEDC),Fuel consumption (economy) - combined (CNG),Fuel consumption (economy) - combined (Ethanol - E85),Fuel consumption (economy) - combined (LPG) (NEDC),"Fuel consumption (economy) - combined (LPG) (NEDC, WLTP equivalent)","Fuel consumption (economy) - combined (CNG) (NEDC, WLTP equivalent)"
0,6.6 l/100 km 35.64 US mpg42.8 UK mpg15.15 km/l,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6.4 l/100 km 36.75 US mpg44.14 UK mpg15.63 km/l,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6.6 l/100 km 35.64 US mpg42.8 UK mpg15.15 km/l,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6.4 l/100 km 36.75 US mpg44.14 UK mpg15.63 km/l,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,6.5 l/100 km 36.19 US mpg43.46 UK mpg15.38 km/l,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
#Getting only the standard(norm) used to calculate the fuel
def norm_fuel_grab(dfserie):
    return " ".join(dfserie.name.split()[3:]).replace('-','').rstrip().lstrip()

#Removing the combined word in fuel
def fuel_clean(dfserie):
    return " ".join(dfserie.split()[0:1]).replace('combined','')

In [29]:
#Building the sum columns with all values adding the norm at the end of the string value
for column in df_fuel_combined.columns:
    df_fuel_combined[column] = df_fuel_combined[column].copy().apply(lambda item: fuel_clean(item) if (not isinstance(item, float)) else item)
    df_fuel_combined[column] = df_fuel_combined[[column]].copy().apply(lambda item: item +' ' +norm_fuel_grab(item)) 
df_fuel_combined['fuel_combined_clean'] = df_fuel_combined.fillna('').sum(axis=1).copy()
df_fuel_combined['fuel_combined_clean'] = df_fuel_combined['fuel_combined_clean'].apply(lambda item: np.nan if len(item) ==0 else item)
df_fuel_combined.head()


C:\Users\UTILIZ~1\AppData\Local\Temp/ipykernel_6272/114773256.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fuel_combined[column] = df_fuel_combined[column].copy().apply(lambda item: fuel_clean(item) if (not isinstance(item, float)) else item)
C:\Users\UTILIZ~1\AppData\Local\Temp/ipykernel_6272/114773256.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fuel_combined[column] = df_fuel_combined[[column]].copy().apply(lambda item: item +' ' +norm_fuel_grab(item))
C:\Users\UTILIZ~1\AppData\Local\

,Fuel consumption (economy) - combined,Fuel consumption (economy) - combined (NEDC),Fuel consumption (economy) - combined (LPG),"Fuel consumption (economy) - combined (NEDC, WLTP equivalent)",Fuel consumption (economy) - combined (CNG) (NEDC),Fuel consumption (economy) - combined (CNG),Fuel consumption (economy) - combined (Ethanol - E85),Fuel consumption (economy) - combined (LPG) (NEDC),"Fuel consumption (economy) - combined (LPG) (NEDC, WLTP equivalent)","Fuel consumption (economy) - combined (CNG) (NEDC, WLTP equivalent)",fuel_combined_clean
0,6.6 combined,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.6 combined
1,6.4 combined,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.4 combined
2,6.6 combined,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.6 combined
3,6.4 combined,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.4 combined
4,NaN,6.5 combined (NEDC),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.5 combined (NEDC)


In [30]:
#Adding the new column with unit in the title name
df_fuel_combined['fuel_combined_clean (l/100km)'] = df_fuel_combined['fuel_combined_clean'].apply(lambda item: item.split()[0] if (not isinstance(item, float)) else item)
df_fuel_combined['fuel_combined_clean (l/100km)'] = df_fuel_combined['fuel_combined_clean (l/100km)'].apply(lambda item: item if (not isinstance(item, float) and item.replace('+','-').find('-') == -1) else item) ##. was float(item)   #and item.isdigit()) else item) #.str.isnumeric()
df_fuel_combined['fuel_combined_clean (l/100km)'] = df_fuel_combined['fuel_combined_clean (l/100km)'].apply(lambda item: basic_split_bin(item) if (not isinstance(item, float)) else item)
df_fuel_combined.head()

C:\Users\UTILIZ~1\AppData\Local\Temp/ipykernel_6272/2280192591.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fuel_combined['fuel_combined_clean (l/100km)'] = df_fuel_combined['fuel_combined_clean'].apply(lambda item: item.split()[0] if (not isinstance(item, float)) else item)
C:\Users\UTILIZ~1\AppData\Local\Temp/ipykernel_6272/2280192591.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fuel_combined['fuel_combined_clean (l/100km)'] = df_fuel_combined['fuel_combined_clean (l/100km)'].apply(lam

,Fuel consumption (economy) - combined,Fuel consumption (economy) - combined (NEDC),Fuel consumption (economy) - combined (LPG),"Fuel consumption (economy) - combined (NEDC, WLTP equivalent)",Fuel consumption (economy) - combined (CNG) (NEDC),Fuel consumption (economy) - combined (CNG),Fuel consumption (economy) - combined (Ethanol - E85),Fuel consumption (economy) - combined (LPG) (NEDC),"Fuel consumption (economy) - combined (LPG) (NEDC, WLTP equivalent)","Fuel consumption (economy) - combined (CNG) (NEDC, WLTP equivalent)",fuel_combined_clean,fuel_combined_clean (l/100km)
0,6.6 combined,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.6 combined,6.6
1,6.4 combined,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.4 combined,6.4
2,6.6 combined,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.6 combined,6.6
3,6.4 combined,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.4 combined,6.4
4,NaN,6.5 combined (NEDC),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.5 combined (NEDC),6.5


In [31]:
# for value in df_fuel_combined['fuel_combined_clean (l/100km)']:
#     if type(value)==float:
#         print(float(value))

In [32]:
# Fixing the weird values in the data
def fixpoints(item):
    if type(item) == str:
        item = item.replace('..', '.')
        item = item.split()[0]
        if len(item)> 5 and item.count('.')==2:
            # item = ".".join([item.split('.')[0],item.split('.')[1][0]]) + ".".join([item.split('.')[1][1:],item.split('.')[2]]
            item = item.split('.')[0]+'.'+item.split('.')[1][0]+ "-"+item.split('.')[1][1:]+'.'+item.split('.')[2]
            # return [i for i in item if i != '.']
            return item
    return item


df_fuel_combined['fuel_combined_clean (l/100km)'] = df_fuel_combined['fuel_combined_clean'].apply(fixpoints)\
.apply(lambda item: [re.findall('\d+\.*\d*',x) for x in item.split('-')] if not isinstance(item,float) else [])\
.apply(lambda item: [i[0] if len(i)>0 else '' for i in item] if len(item)>0 else [])

C:\Users\UTILIZ~1\AppData\Local\Temp/ipykernel_6272/3871185777.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fuel_combined['fuel_combined_clean (l/100km)'] = df_fuel_combined['fuel_combined_clean'].apply(fixpoints)\


### Urban

In [33]:
df_fuel_urban.head()

,Fuel consumption (economy) - urban,Fuel consumption (economy) - urban (NEDC),Fuel consumption (economy) - urban (LPG),"Fuel consumption (economy) - urban (NEDC, WLTP equivalent)",Fuel consumption (economy) - urban (CNG) (NEDC),Fuel consumption (economy) - urban (CNG),Fuel consumption (economy) - urban (Ethanol - E85),Fuel consumption (economy) - urban (LPG) (NEDC),"Fuel consumption (economy) - urban (LPG) (NEDC, WLTP equivalent)","Fuel consumption (economy) - urban (CNG) (NEDC, WLTP equivalent)"
0,9.1 l/100 km 25.85 US mpg31.04 UK mpg10.99 km/l,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8.5 l/100 km 27.67 US mpg33.23 UK mpg11.76 km/l,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,9.1 l/100 km 25.85 US mpg31.04 UK mpg10.99 km/l,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,8.5 l/100 km 27.67 US mpg33.23 UK mpg11.76 km/l,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,8.4 l/100 km 28 US mpg33.63 UK mpg11.9 km/l,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
#Building the sum columns with all values adding the norm at the end of the string value
for column in df_fuel_urban.columns:
    df_fuel_urban[column] = df_fuel_urban[column].copy().apply(lambda item: fuel_clean(item) if (not isinstance(item, float)) else item)
    df_fuel_urban[column] = df_fuel_urban[[column]].copy().apply(lambda item: item +' ' +norm_fuel_grab(item)) 
df_fuel_urban['fuel_urban_clean'] = df_fuel_urban.fillna('').sum(axis=1).copy()
df_fuel_urban['fuel_urban_clean'] = df_fuel_urban['fuel_urban_clean'].apply(lambda item: np.nan if len(item) ==0 else item)


#Adding the new column with unit in the title name
df_fuel_urban['fuel_urban_clean (l/100km)'] = df_fuel_urban['fuel_urban_clean'].apply(lambda item: item.split()[0] if (not isinstance(item, float)) else item)
df_fuel_urban['fuel_urban_clean (l/100km)'] = df_fuel_urban['fuel_urban_clean (l/100km)'].apply(lambda item: item if (not isinstance(item, float) and item.replace('+','-').find('-') == -1) else item) ##. was float(item)   #and item.isdigit()) else item) #.str.isnumeric()
df_fuel_urban['fuel_urban_clean (l/100km)'] = df_fuel_urban['fuel_urban_clean (l/100km)'].apply(lambda item: basic_split_bin(item) if (not isinstance(item, float)) else item)


#Floating every number to be able to do calculation
df_fuel_urban['fuel_urban_clean (l/100km)'].apply(lambda item: fixpoints(item) if not isinstance(item, float) else '')
df_fuel_urban.head()

C:\Users\UTILIZ~1\AppData\Local\Temp/ipykernel_6272/2558895465.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fuel_urban[column] = df_fuel_urban[column].copy().apply(lambda item: fuel_clean(item) if (not isinstance(item, float)) else item)
C:\Users\UTILIZ~1\AppData\Local\Temp/ipykernel_6272/2558895465.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fuel_urban[column] = df_fuel_urban[[column]].copy().apply(lambda item: item +' ' +norm_fuel_grab(item))
C:\Users\UTILIZ~1\AppData\Local\Temp/ipyke

,Fuel consumption (economy) - urban,Fuel consumption (economy) - urban (NEDC),Fuel consumption (economy) - urban (LPG),"Fuel consumption (economy) - urban (NEDC, WLTP equivalent)",Fuel consumption (economy) - urban (CNG) (NEDC),Fuel consumption (economy) - urban (CNG),Fuel consumption (economy) - urban (Ethanol - E85),Fuel consumption (economy) - urban (LPG) (NEDC),"Fuel consumption (economy) - urban (LPG) (NEDC, WLTP equivalent)","Fuel consumption (economy) - urban (CNG) (NEDC, WLTP equivalent)",fuel_urban_clean,fuel_urban_clean (l/100km)
0,9.1 urban,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.1 urban,9.1
1,8.5 urban,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.5 urban,8.5
2,9.1 urban,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.1 urban,9.1
3,8.5 urban,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.5 urban,8.5
4,NaN,8.4 urban (NEDC),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.4 urban (NEDC),8.4


In [35]:
df_fuel_urban['fuel_urban_clean (l/100km)'] = df_fuel_urban['fuel_urban_clean (l/100km)'].apply(fixpoints)\
.apply(lambda item: [re.findall('\d+\.*\d*',x) for x in item.split('-')] if not isinstance(item,(float,list)) else [])\
.apply(lambda item: [i[0] if len(i)>0 else '' for i in item] if len(item)>0 else [])

C:\Users\UTILIZ~1\AppData\Local\Temp/ipykernel_6272/2407303911.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fuel_urban['fuel_urban_clean (l/100km)'] = df_fuel_urban['fuel_urban_clean (l/100km)'].apply(fixpoints)\


### Extra_urban

In [36]:
#Building the sum columns with all values adding the norm at the end of the string value
for column in df_fuel_extra_urban.columns:
    df_fuel_extra_urban[column] = df_fuel_extra_urban[column].copy().apply(lambda item: fuel_clean(item) if (not isinstance(item, float)) else item)
    df_fuel_extra_urban[column] = df_fuel_extra_urban[[column]].copy().apply(lambda item: item +' ' +norm_fuel_grab(item)) 
df_fuel_extra_urban['fuel_extra_urban_clean'] = df_fuel_extra_urban.fillna('').sum(axis=1).copy()
df_fuel_extra_urban['fuel_extra_urban_clean'] = df_fuel_extra_urban['fuel_extra_urban_clean'].apply(lambda item: np.nan if len(item) ==0 else item)

#Adding the new column with unit in the title name
df_fuel_extra_urban['fuel_extra_urban_clean (l/100km)'] = df_fuel_extra_urban['fuel_extra_urban_clean'].apply(lambda item: item.split()[0] if (not isinstance(item, float)) else item)
df_fuel_extra_urban['fuel_extra_urban_clean (l/100km)'] = df_fuel_extra_urban['fuel_extra_urban_clean (l/100km)'].apply(lambda item: item if (not isinstance(item, float) and item.replace('+','-').find('-') == -1) else item) ##. was float(item)   #and item.isdigit()) else item) #.str.isnumeric()
df_fuel_extra_urban['fuel_extra_urban_clean (l/100km)'] = df_fuel_extra_urban['fuel_extra_urban_clean (l/100km)'].apply(lambda item: basic_split_bin(item) if (not isinstance(item, float)) else item)


#Floating every number to be able to do calculation
df_fuel_extra_urban['fuel_extra_urban_clean (l/100km)'].apply(lambda item: fixpoints(item) if not isinstance(item, float) else '')
df_fuel_extra_urban.head()


C:\Users\UTILIZ~1\AppData\Local\Temp/ipykernel_6272/2726853310.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fuel_extra_urban[column] = df_fuel_extra_urban[column].copy().apply(lambda item: fuel_clean(item) if (not isinstance(item, float)) else item)
C:\Users\UTILIZ~1\AppData\Local\Temp/ipykernel_6272/2726853310.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fuel_extra_urban[column] = df_fuel_extra_urban[[column]].copy().apply(lambda item: item +' ' +norm_fuel_grab(item))
C:\Users\UTILIZ~1\

,Fuel consumption (economy) - extra urban,Fuel consumption (economy) - extra urban (NEDC),Fuel consumption (economy) - extra urban (LPG),"Fuel consumption (economy) - extra urban (NEDC, WLTP equivalent)",Fuel consumption (economy) - extra urban (CNG) (NEDC),Fuel consumption (economy) - extra urban (CNG),Fuel consumption (economy) - extra urban (Ethanol - E85),Fuel consumption (economy) - extra urban (LPG) (NEDC),"Fuel consumption (economy) - extra urban (LPG) (NEDC, WLTP equivalent)","Fuel consumption (economy) - extra urban (CNG) (NEDC, WLTP equivalent)",fuel_extra_urban_clean,fuel_extra_urban_clean (l/100km)
0,5.2 extra urban,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.2 extra urban,5.2
1,5.1 extra urban,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.1 extra urban,5.1
2,5.1 extra urban,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.1 extra urban,5.1
3,5.1 extra urban,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.1 extra urban,5.1
4,NaN,5.4 extra urban (NEDC),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.4 extra urban (NEDC),5.4


In [37]:
df_fuel_extra_urban['fuel_extra_urban_clean (l/100km)'] = df_fuel_extra_urban['fuel_extra_urban_clean (l/100km)'].apply(fixpoints)\
.apply(lambda item: [re.findall('\d+\.*\d*',x) for x in item.split('-')] if not isinstance(item,(float, list)) else [])\
.apply(lambda item: [i[0] if len(i)>0 else '' for i in item] if len(item)>0 else [])

C:\Users\UTILIZ~1\AppData\Local\Temp/ipykernel_6272/65901365.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fuel_extra_urban['fuel_extra_urban_clean (l/100km)'] = df_fuel_extra_urban['fuel_extra_urban_clean (l/100km)'].apply(fixpoints)\


## Final dataframe

In [38]:
#First merge
df_final = df_gev_co2.merge(df_fuel_combined[['fuel_combined_clean','fuel_combined_clean (l/100km)']],left_index=True,right_index=True)
df_final = df_final.merge(df_fuel_urban[['fuel_urban_clean','fuel_urban_clean (l/100km)']],left_index=True,right_index=True)
df_final = df_final.merge(df_fuel_extra_urban[['fuel_extra_urban_clean','fuel_extra_urban_clean (l/100km)']],left_index=True,right_index=True)
df_final.head()

,Brand,Model,Generation,Modification (Engine),Body type,Fuel tank capacity,Fuel consumption (economy) - combined,Fuel Type,CO2 emissions,Power,co2_clean,co2_nb,co2_unit,Fuel_tank_capacity_clean_(liters),fuel_combined_clean,fuel_combined_clean (l/100km),fuel_urban_clean,fuel_urban_clean (l/100km),fuel_extra_urban_clean,fuel_extra_urban_clean (l/100km)
0,Abarth,124 Spider,124 GT,1.4 MultiAir (170 Hp) Automatic,Roadster,45 l 11.89 US gal | 9.9 UK gal,6.6 l/100 km 35.64 US mpg42.8 UK mpg15.15 km/l,Petrol (Gasoline),153 g/km,170 Hp @ 5500 rpm.,153 g/km,153.0,g/km,45.0,6.6 combined,[6.6],9.1 urban,[9.1],5.2 extra urban,[5.2]
1,Abarth,124 Spider,124 GT,1.4 MultiAir (170 Hp),Roadster,45 l 11.89 US gal | 9.9 UK gal,6.4 l/100 km 36.75 US mpg44.14 UK mpg15.63 km/l,Petrol (Gasoline),148 g/km,170 Hp @ 5500 rpm.,148 g/km,148.0,g/km,45.0,6.4 combined,[6.4],8.5 urban,[8.5],5.1 extra urban,[5.1]
2,Abarth,124 Spider,124 Spider,1.4 MultiAir (170 Hp) Automatic,Cabriolet,45 l 11.89 US gal | 9.9 UK gal,6.6 l/100 km 35.64 US mpg42.8 UK mpg15.15 km/l,Petrol (Gasoline),153 g/km,170 Hp @ 5500 rpm.,153 g/km,153.0,g/km,45.0,6.6 combined,[6.6],9.1 urban,[9.1],5.1 extra urban,[5.1]
3,Abarth,124 Spider,124 Spider,1.4 MultiAir (170 Hp),Cabriolet,45 l 11.89 US gal | 9.9 UK gal,6.4 l/100 km 36.75 US mpg44.14 UK mpg15.63 km/l,Petrol (Gasoline),148 g/km,170 Hp @ 5500 rpm.,148 g/km,148.0,g/km,45.0,6.4 combined,[6.4],8.5 urban,[8.5],5.1 extra urban,[5.1]
4,Abarth,500,500C,1.4 T-Jet (140 Hp) MTA,Cabriolet,35 l 9.25 US gal | 7.7 UK gal,NaN,Petrol (Gasoline),NaN,140 Hp @ 5500 rpm.,151 g/km(NEDC),151.0,g/km(NEDC),35.0,6.5 combined (NEDC),[6.5],8.4 urban (NEDC),[8.4],5.4 extra urban (NEDC),[5.4]


In [39]:
df_gev_beev = df_final[['Brand','Model','Generation','Modification (Engine)','Body type','Fuel Type','Power','co2_nb','co2_unit','Fuel_tank_capacity_clean_(liters)','fuel_combined_clean','fuel_combined_clean (l/100km)','fuel_urban_clean','fuel_urban_clean (l/100km)','fuel_extra_urban_clean', 'fuel_extra_urban_clean (l/100km)']]
df_gev_beev.head()

,Brand,Model,Generation,Modification (Engine),Body type,Fuel Type,Power,co2_nb,co2_unit,Fuel_tank_capacity_clean_(liters),fuel_combined_clean,fuel_combined_clean (l/100km),fuel_urban_clean,fuel_urban_clean (l/100km),fuel_extra_urban_clean,fuel_extra_urban_clean (l/100km)
0,Abarth,124 Spider,124 GT,1.4 MultiAir (170 Hp) Automatic,Roadster,Petrol (Gasoline),170 Hp @ 5500 rpm.,153.0,g/km,45.0,6.6 combined,[6.6],9.1 urban,[9.1],5.2 extra urban,[5.2]
1,Abarth,124 Spider,124 GT,1.4 MultiAir (170 Hp),Roadster,Petrol (Gasoline),170 Hp @ 5500 rpm.,148.0,g/km,45.0,6.4 combined,[6.4],8.5 urban,[8.5],5.1 extra urban,[5.1]
2,Abarth,124 Spider,124 Spider,1.4 MultiAir (170 Hp) Automatic,Cabriolet,Petrol (Gasoline),170 Hp @ 5500 rpm.,153.0,g/km,45.0,6.6 combined,[6.6],9.1 urban,[9.1],5.1 extra urban,[5.1]
3,Abarth,124 Spider,124 Spider,1.4 MultiAir (170 Hp),Cabriolet,Petrol (Gasoline),170 Hp @ 5500 rpm.,148.0,g/km,45.0,6.4 combined,[6.4],8.5 urban,[8.5],5.1 extra urban,[5.1]
4,Abarth,500,500C,1.4 T-Jet (140 Hp) MTA,Cabriolet,Petrol (Gasoline),140 Hp @ 5500 rpm.,151.0,g/km(NEDC),35.0,6.5 combined (NEDC),[6.5],8.4 urban (NEDC),[8.4],5.4 extra urban (NEDC),[5.4]


In [40]:
### Function to get rid of list values and make them all float type
def remove_list_fuel(value):
    if type(value) == list:
        if len(value) == 1 or '' in value:
            value = value[0]
            return float(value)
        elif len(value) == 2:
            return(sum([float(x)/2 for x in value]))
    else:
        return float(value)

### Same but for fuel tank
def remove_list_tank(item):
    if type(item) == list:
        if len(item) == 2:
            return(sum([float(x)/2 for x in item]))
    else:
        return float(item)
            
            
df_gev_beev['fuel_combined_clean (l/100km)'] = df_gev_beev['fuel_combined_clean (l/100km)'].apply(lambda x : remove_list_fuel(x))
df_gev_beev['Fuel_tank_capacity_clean_(liters)'] = df_gev_beev['Fuel_tank_capacity_clean_(liters)'].apply(lambda x : remove_list_tank(x))
df_gev_beev['fuel_urban_clean (l/100km)'] = df_gev_beev['fuel_urban_clean (l/100km)'].apply(lambda x : remove_list_fuel(x))
df_gev_beev['fuel_extra_urban_clean (l/100km)'] = df_gev_beev['fuel_extra_urban_clean (l/100km)'].apply(lambda x : remove_list_fuel(x))

C:\Users\UTILIZ~1\AppData\Local\Temp/ipykernel_6272/722802727.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gev_beev['fuel_combined_clean (l/100km)'] = df_gev_beev['fuel_combined_clean (l/100km)'].apply(lambda x : remove_list_fuel(x))
C:\Users\UTILIZ~1\AppData\Local\Temp/ipykernel_6272/722802727.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gev_beev['Fuel_tank_capacity_clean_(liters)'] = df_gev_beev['Fuel_tank_capacity_clean_(liters)'].apply(lambda x : remove_list_tank(x))
C:\Users\UTILI

In [41]:
##### Renaming columns and calculating required fields      ###########


df_gev_beev.rename(columns={'co2_nb' : 'CO2 emissions (g/km)', 'Fuel_tank_capacity_clean_(liters)': 'Fuel tank capacity (l)', 'fuel_combined_clean (l/100km)':'Fuel consumption - combined (l/100km)'
, 'fuel_urban_clean (l/100km)':'Fuel consumption - urban (l/100km)', 'fuel_extra_urban_clean (l/100km)': 'Fuel consumption - extra urban (l/100km)'}, inplace = True)
df_gev_beev['Power (hp)'] = df_gev_beev['Power'].apply(lambda x: str(x).split(' ')[0])


df_gev_beev['Range (km)'] = df_gev_beev[['Fuel tank capacity (l)','Fuel consumption - combined (l/100km)']].apply(lambda x : float(x[0])/float(x[1])*100, axis = 1)
df_gev_beev['Range (km)'] = df_gev_beev[['Fuel tank capacity (l)', 'Fuel consumption - urban (l/100km)','Fuel consumption - extra urban (l/100km)', 'Range (km)']].apply(lambda x : x[0]/((x[1] + x[2])/2)*100 if x[3] == np.nan else x[3], axis = 1)

C:\Users\Utilizador\anaconda3\envs\Safe space\lib\site-packages\pandas\core\frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
C:\Users\UTILIZ~1\AppData\Local\Temp/ipykernel_6272/1934671949.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gev_beev['Power (hp)'] = df_gev_beev['Power'].apply(lambda x: str(x).split(' ')[0])
C:\Users\UTILIZ~1\AppData\Local\Temp/ipykernel_6272/1934671949.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = valu

In [42]:
df_gev_beev.fillna('N/A', inplace = True)

C:\Users\Utilizador\anaconda3\envs\Safe space\lib\site-packages\pandas\core\frame.py:4462: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


In [43]:
df_gev_beev['Range (km)'] = df_gev_beev['Range (km)'].apply(lambda x : int(x) if x != 'N/A' else x) 

C:\Users\UTILIZ~1\AppData\Local\Temp/ipykernel_6272/3209688464.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gev_beev['Range (km)'] = df_gev_beev['Range (km)'].apply(lambda x : int(x) if x != 'N/A' else x)


In [44]:
df_gev_beev.head()

,Brand,Model,Generation,Modification (Engine),Body type,Fuel Type,Power,CO2 emissions (g/km),co2_unit,Fuel tank capacity (l),fuel_combined_clean,Fuel consumption - combined (l/100km),fuel_urban_clean,Fuel consumption - urban (l/100km),fuel_extra_urban_clean,Fuel consumption - extra urban (l/100km),Power (hp),Range (km)
0,Abarth,124 Spider,124 GT,1.4 MultiAir (170 Hp) Automatic,Roadster,Petrol (Gasoline),170 Hp @ 5500 rpm.,153.0,g/km,45.0,6.6 combined,6.6,9.1 urban,9.1,5.2 extra urban,5.2,170,681
1,Abarth,124 Spider,124 GT,1.4 MultiAir (170 Hp),Roadster,Petrol (Gasoline),170 Hp @ 5500 rpm.,148.0,g/km,45.0,6.4 combined,6.4,8.5 urban,8.5,5.1 extra urban,5.1,170,703
2,Abarth,124 Spider,124 Spider,1.4 MultiAir (170 Hp) Automatic,Cabriolet,Petrol (Gasoline),170 Hp @ 5500 rpm.,153.0,g/km,45.0,6.6 combined,6.6,9.1 urban,9.1,5.1 extra urban,5.1,170,681
3,Abarth,124 Spider,124 Spider,1.4 MultiAir (170 Hp),Cabriolet,Petrol (Gasoline),170 Hp @ 5500 rpm.,148.0,g/km,45.0,6.4 combined,6.4,8.5 urban,8.5,5.1 extra urban,5.1,170,703
4,Abarth,500,500C,1.4 T-Jet (140 Hp) MTA,Cabriolet,Petrol (Gasoline),140 Hp @ 5500 rpm.,151.0,g/km(NEDC),35.0,6.5 combined (NEDC),6.5,8.4 urban (NEDC),8.4,5.4 extra urban (NEDC),5.4,140,538


In [45]:
df_gev_beev['Range (km)'].value_counts(dropna = False)

N/A     15415
1000      452
833       338
714       282
666       251
        ...  
420         1
1475        1
421         1
1481        1
376         1
Name: Range (km), Length: 1123, dtype: int64

In [46]:
### Function to convert every value in CO2 emissions column to float
def float_co2(value):
    if type(value) == str and value != 'N/A' and value!="LEV3":
        return(float(value))
    elif value == 'LEV3':
        return 'N/A'
    else:
        return value                       

df_gev_beev['CO2 emissions (g/km)'] = df_gev_beev['CO2 emissions (g/km)'].apply(lambda item : float_co2(item))
df_gev_beev['Power (hp)'] = df_gev_beev['Power (hp)'].apply(lambda item : float_co2(item))  # Applying the same for Power column

C:\Users\UTILIZ~1\AppData\Local\Temp/ipykernel_6272/492017010.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gev_beev['CO2 emissions (g/km)'] = df_gev_beev['CO2 emissions (g/km)'].apply(lambda item : float_co2(item))
C:\Users\UTILIZ~1\AppData\Local\Temp/ipykernel_6272/492017010.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gev_beev['Power (hp)'] = df_gev_beev['Power (hp)'].apply(lambda item : float_co2(item))  # Applying the same for Power column


In [47]:
# CV = (CO2 : 45) + (P : 40) x 1,6  --------------------- (1 ch = 0,736 kW)
df_gev_beev['CV'] = df_gev_beev[['CO2 emissions (g/km)', 'Power (hp)']].apply(lambda item: item[0]/45 + item[1]*0.736 / 40 * 1.6 if type(item[0]) != str and type(item[1]) != str else 'N/A', axis = 1)

C:\Users\UTILIZ~1\AppData\Local\Temp/ipykernel_6272/611983789.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gev_beev['CV'] = df_gev_beev[['CO2 emissions (g/km)', 'Power (hp)']].apply(lambda item: item[0]/45 + item[1]*0.736 / 40 * 1.6 if type(item[0]) != str and type(item[1]) != str else 'N/A', axis = 1)


In [48]:
df_gev_beev['CV'].fillna('N/A', inplace = True)
df_gev_beev['CV'] = df_gev_beev['CV'].apply(lambda item : int(item) if type(item) != str else item)

C:\Users\Utilizador\anaconda3\envs\Safe space\lib\site-packages\pandas\core\series.py:4463: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(
C:\Users\UTILIZ~1\AppData\Local\Temp/ipykernel_6272/1310235633.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gev_beev['CV'] = df_gev_beev['CV'].apply(lambda item : int(item) if type(item) != str else item)


In [49]:
df_gev_beev['Fuel Type'].value_counts()

Petrol (Gasoline)               31074
Diesel                          11659
Petrol / CNG                      126
Petrol / Ethanol - E85            123
Petrol / LPG                      114
Mixture of two stroke engine       12
LPG                                 4
Name: Fuel Type, dtype: int64

In [50]:
#Dictionary for categories

dicto_cat = {0 : 'SUV', 1 : 'Berline', 2 : 'Citadine', 3 : 'Utilitaire', 4 : 'Compacte', 5 : 'Van', 6 : 'ALL'}

dicto_gev = {'Sedan': 1 , 'Hatchback': 2 , 'Station wagon (estate)':1, 'Coupe':1, 'SUV':0,
       'Minivan':3, 'Pick-up':0, 'Cabriolet':4, 'Off-road vehicle':0, 'Minivan, MPV':3,
       'MPV':0, 'SUV, Crossover':0, 'Crossover':0, 'Roadster':1, 'Liftback':1,
       'Fastback':1, 'Station wagon (estate), Crossover':1, 'Coupe - Cabriolet':4,
       'Targa':1, 'Grand Tourer':3, 'Off-road vehicle, SUV':0, 'Coupe, Liftback':0,
       'Coupe, SUV':0, 'Cabriolet, Coupe':4, 'Station wagon (estate), MPV':0,
       'Off-road vehicle, Cabriolet':0, 'Minivan, Crossover':3,
       'Hatchback, Crossover':0, 'SAC':6, 'N/A':6, 'SAV':0, 'Cabriolet, SUV':0,
       'Coupe, Hatchback':4, 'Coupe - Cabriolet, Roadster':4, 'SUV, MPV':0,
       'Sedan, Fastback':1, 'Off-road vehicle, Cabriolet, SUV':0,
       'Coupe, Crossover':0, 'Off-road vehicle, Station wagon (estate)':1,
       'Off-road vehicle, Pick-up':0, 'Minivan, Station wagon (estate)':3,
       'Cabriolet, Roadster':4, 'Cabriolet, Hatchback':4, 'Coupe, Fastback':1,
       'Crossover, Liftback':0, 'CUV':0, 'Hatchback, Fastback':4, 'Pick-up, SUV':0,
       'Cabriolet, Crossover':0}

In [51]:
df_categories = pd.DataFrame(df_gev_beev['Body type'].value_counts()).T.reset_index()
df_categories.columns

Index(['index', 'Sedan', 'Hatchback', 'Station wagon (estate)', 'Coupe', 'SUV',
       'Minivan', 'Pick-up', 'Cabriolet', 'Off-road vehicle', 'Minivan, MPV',
       'MPV', 'SUV, Crossover', 'Crossover', 'Roadster', 'Liftback',
       'Fastback', 'Station wagon (estate), Crossover', 'Coupe - Cabriolet',
       'Targa', 'Grand Tourer', 'Off-road vehicle, SUV', 'Cabriolet, Coupe',
       'Coupe, Liftback', 'Coupe, SUV', 'Station wagon (estate), MPV',
       'Off-road vehicle, Cabriolet', 'Minivan, Crossover',
       'Hatchback, Crossover', 'SAC', 'N/A', 'SAV', 'Cabriolet, SUV',
       'Coupe, Hatchback', 'SUV, MPV', 'Coupe - Cabriolet, Roadster',
       'Sedan, Fastback', 'Coupe, Crossover',
       'Off-road vehicle, Cabriolet, SUV',
       'Off-road vehicle, Station wagon (estate)', 'Off-road vehicle, Pick-up',
       'Minivan, Station wagon (estate)', 'Cabriolet, Roadster', 'CUV',
       'Crossover, Liftback', 'Cabriolet, Hatchback', 'Hatchback, Fastback',
       'Coupe, Fastback', 'Cab

In [52]:
## Function to convert the categories
def category_conversion(item):
    for keys in dicto_gev.keys():
        if item == keys:
            return dicto_cat[dicto_gev[keys]]

In [53]:
df_gev_beev['Category'] = df_gev_beev['Body type'].apply(lambda item : category_conversion(item))

In [54]:
### This is the dataframe we will use in our calculations, with no missing values
df_gev_beev2 = df_gev_beev[['Brand', 'Model', 'Modification (Engine)',
       'Fuel Type', 'CO2 emissions (g/km)', 'Fuel tank capacity (l)',
       'Fuel consumption - combined (l/100km)', 'Fuel consumption - urban (l/100km)',
       'Fuel consumption - extra urban (l/100km)', 'Power (hp)', 'Range (km)',
       'CV', 'Category']]
df_gev_beev2.replace('N/A', np.nan, inplace = True)
df_gev_beev2.dropna(inplace = True)
df_gev_beev2.isna().sum()

C:\Users\Utilizador\anaconda3\envs\Safe space\lib\site-packages\pandas\core\frame.py:4524: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(
C:\Users\UTILIZ~1\AppData\Local\Temp/ipykernel_6272/2069414648.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gev_beev2.dropna(inplace = True)


Brand                                       0
Model                                       0
Modification (Engine)                       0
Fuel Type                                   0
CO2 emissions (g/km)                        0
Fuel tank capacity (l)                      0
Fuel consumption - combined (l/100km)       0
Fuel consumption - urban (l/100km)          0
Fuel consumption - extra urban (l/100km)    0
Power (hp)                                  0
Range (km)                                  0
CV                                          0
Category                                    0
dtype: int64

In [55]:
#### CHeck the LPG and drop those values

# Petrol (Gasoline)               31074
# Diesel                          11659

# Petrol / CNG                      126
# Petrol / Ethanol - E85            123
# Petrol / LPG                      114

# Mixture of two stroke engine       12
# LPG                                 4


#All Petrol convert to Regular gasoline
#Drop Mixture and LPG

In [56]:
df_gev_beev.to_csv("df_gev_beev.csv",index = False)
df_gev_beev2.to_csv("df_gev_beev2.csv",index = False)